# Lab Course Machine Learning
## Exercise Sheet 7
#### Prof. Dr. Dr. Lars Schmidt-Thieme, Hadi Samer Jomaa
#### Information Systems and Machine Learning Lab
#### University of Hildesheim
December 11th, 2017
Submission on December 18th, 2017 at 8:00 am, (on moodle, course code 3113)

## What is k-Nearest Neighbors

The model for kNN is the entire training dataset. 
When a prediction is required for a unseen data instance, the kNN algorithm will search through the training dataset for the k-most similar instances. The prediction attribute of the most similar instances is summarized and returned as the prediction for the unseen instance.
The similarity measure is dependent on the type of data. For real-valued data, the Euclidean distance can be used. Other other types of data such as categorical or binary data, Hamming distance can be used.
### How does k-Nearest Neighbors Work
kNN is a powerful tool of prediction because it does not assume anything about the data, other than a distance measure can be calculated consistently between any two instances. As such, it is called non-parametric or non-linear as it does not assume a functional form.
The kNN algorithm is belongs to the family of:
#### Instance-based  algorithms 
Model the problem using data instances (or rows) in order to make predictive decisions. In the  kNN algorithm all training observations are retained as part of the model.
#### Competitive learning  algorithms 
Internally uses competition between model elements (data instances) in order to make a predictive decision. The objective similarity measure between data instances causes each data instance to compete to “win” or be most similar to a given unseen data instance and contribute to a prediction.
#### Lazy learning algorithms 
Does not build a model until the time that a prediction is required.  A disadvantage is that it can be computationally expensive to repeat the same or similar searches over larger training datasets.

# Exercises
### Datasets

1. Classification Datasets: You can use one of the two datasets ( or optionally, both datasets).
   
       (a) Iris dataset D1 : Target attribute class:{Iris Setosa, Iris Versicolour, Iris Virginica}. https://archive.ics.uci.edu/ml/datasets/Iris

        (b) Wine Quality dataset D2 : Target attribute quality:{0 to 10}. https://archive.ics.uci.edu/ml/datasets/Wine+Quality



In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import operator

Iris = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",header=None)
WWN = pd.read_table("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv",sep=";")

#Showing the Wine data
WWN.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
#Showing the Iris Data
Iris.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


# Exercise 1: Implement K-Nearest Neighbor (KNN) (10 Points)

Your task is to implement KNN algorithm. To implement KNN you have to:

• Split data into a train and a test split (70% and 30% respectively).


In [635]:
def split(Dataframe,P): 
    #Defining the split, the objetive data is kept in the same matrix, at the end (the "-1" values)
    Y = np.array([Dataframe.iloc[:,-1]]).T                        
    Dataframe = Dataframe.drop(Dataframe.columns[[-1,]], axis=1)  
    X = np.array(pd.get_dummies(Dataframe))            
    Matrix = np.concatenate((X,Y),axis =1)                        
    msk = np.random.rand(len(Matrix)) < P                         
    tr = Matrix[msk]
    tst = Matrix[~msk]    
    return tr, tst                                                

#Wine
Wxtr, Wxtst = split(WWN,0.7)

#Iris
Ixtr, Ixtst = split(Iris,0.7)


• Implement a similarity (or a distance) measure. To begin with you can implement the Euclidean
Distance.


In [628]:
def EuclDist(VectorA, VectorB,Long):
    #Defining euclidean distance 
    Dist = []
    for i in np.arange(Long):
        D = (VectorA[i]   - VectorB[i])**2
        Dist.append(D)
    Euclidean = np.sum(Dist)**0.5
    return Euclidean

• Implement a function that returns top K Nearest Neighbors for a given query (data point).

• You should provide the prediction for a given query (for a classification task you can use majority
voting and for a regression you can use mean).

In [630]:
#This function return the k-nearest neighbors using Euclidean Distance
def Neighboor(TrainFrame,TestVector,k,Type):
    Ngh = []
    classVotes = {}
    Long = len(TestVector)-1
    for n in np.arange(len(TrainFrame)):
        D = EuclDist(TrainFrame[n],TestVector,Long)
        F = [D,TrainFrame[n][-1]]
        Ngh.append(F)
    Ngh.sort(key=operator.itemgetter(0))
    Neighbors = np.array(Ngh)[0:k]                      #list of K top results
    #print (Neighbors)                                   #Deactivated for the main program
    if Type=="Class":
        for x in range(len(Neighbors)):
            response = Neighbors[x][-1]
            if response in classVotes:
                classVotes[response] += 1
            else:
                classVotes[response] = 1
        sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
        return sortedVotes[0][0]
    if Type=="Reg":
        response = np.average(Neighbors[:,-1])
        return response

In [486]:
#Example of a list of 10 neighbours -  "Wine Data Set"
#It is observed that the count for this test is 7 for a clasification of 5 vs 3 for a classification of 6

Neighboor(Wxtr,Wxtst[1],10,"Class")

[[ 1.59069829  5.        ]
 [ 2.34581428  5.        ]
 [ 2.37469603  5.        ]
 [ 2.40429283  5.        ]
 [ 2.50754868  5.        ]
 [ 2.50754868  5.        ]
 [ 2.51736867  5.        ]
 [ 2.88879079  6.        ]
 [ 2.88879079  6.        ]
 [ 3.22109749  6.        ]]


5.0

• Measure the quality of your prediction. [Hint: You have to choose a quality criterion according to
the task you are solving i.e. a regression or a classification task].

In [25]:
#The previous program only returns K-NN for a single test vector.
#The program bellow compare a whole Test of vector test, vs the Train set
          #For Regresion, the accuracy method choosen was RMSE
          #For classification the accuracy is measured as: correct clasifications/total clasifications

def LearnKNN(Train,Test,k,Type):
    Prediction = []
    for i in np.arange(len(Test)):
        TestVector= Test[i]
        P =Neighboor(Train,TestVector,k,Type)
        Prediction.append(P)
    if Type=="Reg":
        RMSE = np.sum((Test[:,-1]-Prediction)**2)**0.5
        return RMSE
    if Type=="Class":
        ŷ = 0
        for n in np.arange(len(Prediction)):
            if Prediction[n] == Test[:,-1][n]:
                ŷ = ŷ+1
        Accuracy = ŷ/len(Prediction)*100.0
        return Accuracy

In [417]:
#Example: iris set doing a classification using 1-NN  
#Note: This data set cannot do a regression, due the categorical nature of the objetive data

LearnKNN(Ixtr,Ixtst,1,"Class")

97.82608695652173

In [391]:
#Example: Wine data doing a clasification using 10-NN

LearnKNN(Wxtr,Wxtst,10,"Class")

48.51351351351351

In [631]:
#Example: Wine data doing a regression using 10-NN
#In this case, a regression performs 18% lower than a clasification

LearnKNN(Wxtr,Wxtst,10,"Reg")

30.714003320960945

## Effect of normalization 

   When the units of measure differ between attributes, it is possible for attributes to dominate in their contribution to the distance measure. For the wine data set, the data was normalized ant then tested againg with the KNN-algorithm.


In [10]:
def Normalize(Matrix): #For normalize an array of data
    Matrix = (Matrix - np.average(Matrix,axis=0))/np.std(Matrix,axis=0)
    return Matrix

def split2(Dataframe): #Assume that the last column always will contain the "Y values and that values in "Y" are NOT strings
    Y = np.array([Dataframe.iloc[:,-1]]).T                        #Converting the Y column into array
    Dataframe = Dataframe.drop(Dataframe.columns[[-1,]], axis=1)  #Getting dummies and converting X into array
    X = Normalize(np.array(pd.get_dummies(Dataframe)))            #Normalizing data  
    Matrix = np.concatenate((X,Y),axis =1)                        #We get X and Y together again to do the split
    msk = np.random.rand(len(Matrix)) < 0.8                       #Random assign
    tr = Matrix[msk]
    tst = Matrix[~msk]    
    return tr, tst                                             #Tr is for training, Tst is for testing

In [398]:
Wtr, Wtst = split2(WWN)

#Example: Wine data clasification with normalized data perfoms almost 10% better than non-normalized data

LearnKNN(Wtr,Wtst,10,"Class")

57.91106514994829

# Exercise 2: Optimize and Compare KNN algorithm. (10 Points)

### Part A: (5 Points): Determine Optimal Value of K in KNN algorithm. 
In this exercise you have to provide the optimal value of K for given datasets.

    • How you can choose value of K for KNN? 
             Give a criterion to choose an optimal value of K
    • Implement the criterion for choosing the optimal value of K.
    • Experimentally, give evidence that your chosen value is better than other values of K. 
         [Hint: run your experiment with different values of K and plot the error measure for each value].


 When evaluating different settings (“hyperparameters”) there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. 
 A solution to this problem is a procedure called cross-validation (CV for short). In the basic approach, called k-fold CV, the training set is split into k smaller sets; then the following procedure is followed for each of the k “folds”:

    a) A model is trained using of the folds as training data 
    
    b) The resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).
    
    c) The performance measure reported by k-fold cross-validation is then the average of the values computed in the 
    loop.

In [641]:
#Implementing k-Fold

#List of hyperparameters (In this case, only K)
KNNList = [1,3,5,10,20]

def KFoldKNN(Data,fold,kNN,Type):
    Error = []
    for k in np.arange(fold):
        L = np.random.shuffle(Data)
        w=np.round(len(Data)//fold)
        if k == 0:
            tst = Data[(k*w):((k+1)*w):]
            tr  = Data[((k+1)*w):len(Data):]
        else:
            if k== np.arange(fold)[-1]:
                tst = Data[(k*w):((k+1)*w):]
                tr  = Data[0:(k*w):]
            else:
                tst = Data[(k*w):((k+1)*w):]
                tr  = np.concatenate((Data[0:(k*w):],Data[((k+1)*w):len(Data):]),axis=0)
        Accuracy = LearnKNN(tst,tr,kNN,Type)
        Error.append(Accuracy)
    #print("MAError: ", np.average(Error))                 #Deactivated in main program
    return np.average(Error)

                    

In [439]:
#Example, 3-Fold, using 5-NN, for a "Classification" on the Iris training
#The average value of 3-Folds is presented

KFoldKNN(Ixtr,3,5,"Class")

MAError:  93.7055981813


[92.7536231884058, 91.30434782608695, 97.05882352941177]

In [639]:
#Implementation of a Crossvalidator algorithm
#It will test a list of hyperparameters and return the k-Fold average value of each

def KNNCvalidator(Data,fold,KNNList,Type):
    Scores = []
    for i in np.arange(len(KNNList)):
        KNN = KNNList[i]
        iScore = KFoldKNN(Data,fold,KNN,Type)
        Label = "KNN :",KNNList[i]
        Set = [Label,iScore]
        Scores.append(Set)
    #Best Score
    Scores.sort(key=operator.itemgetter(1))
    print("Best KNN is:")
    print(Scores[-1])
    return Scores
        

In [479]:
#Example cross-validation of different K-NN using 5-Folds, in a Classification of the Iris data
KNNCvalidator(Ixtr,5,KNNList,"Class")

Best KNN is:
[('KNN :', 1), 91.734939759036152]


[[('KNN :', 20), 50.105421686746993],
 [('KNN :', 10), 72.168674698795172],
 [('KNN :', 5), 87.692771084337352],
 [('KNN :', 3), 87.906626506024082],
 [('KNN :', 1), 91.734939759036152]]

In [480]:
#Example cross-validation of different K-NN using 5-Folds, in a Classification of the Wine data
KNNCvalidator(Wxtr,5,KNNList,"Class")

Best KNN is:
[('KNN :', 20), 43.82681256206488]


[[('KNN :', 3), 41.471500343949835],
 [('KNN :', 1), 41.888359506532375],
 [('KNN :', 5), 42.144654323730393],
 [('KNN :', 10), 42.81027085045276],
 [('KNN :', 20), 43.82681256206488]]

In [642]:
#Example cross-validation of different K-NN using 5-Folds, in a Classification of the Normalized- Wine data
KNNCvalidator(Wtr,5,KNNList,"Class")

Best KNN is:
[('KNN :', 20), 51.870033744673513]


[[('KNN :', 1), 48.404997660040884],
 [('KNN :', 3), 49.122490701741427],
 [('KNN :', 5), 50.390397546737603],
 [('KNN :', 10), 51.005498903913896],
 [('KNN :', 20), 51.870033744673513]]

### Part B: (5 Points): Compare KNN algorithm with Tree based method. 
In this task you are allowed to use scikit learn. In particular you have to use Nearest Neighbor and Decision Tree implementation provided
by scikit learn.

    • You should be able to use Nearest Neighbor and Decision Tree provided by scikit learn to solve classification 
    task for two datasets.
    • You have to provide the optimal hyperparameters for both the methods. 
    [Hint: use Grid Search and cross validation and present results for them to support your solution].
    • Present the comparison of the two methods using evaluation results on test datasets. 
        [Hint: Better to use cross validation to ascertain your results]

### KNN Algorithm


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sklearn
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

def splitDf(Dataframe): #Assume that the last column always will contain the "Y values and that values in "Y" are NOT strings
    Y = np.array([Dataframe.iloc[:,-1]]).T                        #Converting the Y column into array
    Dataframe = Dataframe.drop(Dataframe.columns[[-1,]], axis=1)  #Getting dummies and converting X into array
    X = Normalize(np.array(pd.get_dummies(Dataframe)))            #Normalizing data  
    Matrix = np.concatenate((X,Y),axis =1)                        #We get X and Y together again to do the split
    msk = np.random.rand(len(Matrix)) < 0.8                       #Random assign
    tr = Matrix[msk]
    tst = Matrix[~msk]    
    Ytr = np.array([tr[:,-1]]).T
    Xtr= tr[:,0:-1]
    Ytst = np.array([tst[:,-1]]).T
    Xtst= tst[:,0:-1]
    return Xtr, Ytr, Xtst, Ytst                                   #Tr is for training, Tst is for testing

### Iris Data Set - KNN

In [12]:
#Spliting Iris data set
Ixtr,Iytr,Ixtst,Iytst = splitDf(Iris)

In [30]:
#Using KNN Algorithm from Sci-Kit
#List of neighboors is from 1 to 21
KNNParameters= {"n_neighbors":range(1,21,1)}
KNN = sklearn.neighbors.KNeighborsClassifier()

#Grid Search
KNNGrid = GridSearchCV(KNN,KNNParameters)
KNNGrid.fit(Ixtr,Iytr.ravel())
results = pd.DataFrame(KNNGrid.cv_results_)
print("Best KNN value: ",KNNGrid.best_estimator_.n_neighbors)
print("Score: ",KNNGrid.best_score_)

#Results of the GridSearch
results.iloc[:,2:7]

Best KNN value:  3
Score:  0.967479674797


,mean_test_score,mean_train_score,param_n_neighbors,params,rank_test_score
0,0.951220,1.000000,1,{'n_neighbors': 1},8
1,0.951220,0.979772,2,{'n_neighbors': 2},8
2,0.967480,0.979772,3,{'n_neighbors': 3},1
3,0.967480,0.967476,4,{'n_neighbors': 4},1
4,0.959350,0.971492,5,{'n_neighbors': 5},5
5,0.959350,0.963510,6,{'n_neighbors': 6},5
6,0.951220,0.963411,7,{'n_neighbors': 7},8
7,0.959350,0.967526,8,{'n_neighbors': 8},5
8,0.967480,0.971541,9,{'n_neighbors': 9},1
9,0.967480,0.963410,10,{'n_neighbors': 10},1


In [23]:
#Cross Validation of KNN- Folds=5
KNNClf = sklearn.neighbors.KNeighborsClassifier(n_neighbors=3)
CVKNN =  cross_val_score(KNNClf,Ixtst,Iytst.ravel(),cv=5)
CVKNN

array([ 1. ,  1. ,  0.8,  0.8,  1. ])

In [118]:
#Results on Test data of the best Hyperparameter found

KNNClf.fit(Ixtr,Iytr.ravel())
KNNClf.score(Ixtst,Iytst)

0.92592592592592593

### Wine Data Set - KNN

In [13]:
#Spliting Iris data set
Wxtr,Wytr,Wxtst,Wytst = splitDf(WWN)

In [14]:
#Using KNN Algorithm from Sci-Kit
#List of neighboors is from 1 to 21
KNNParameters= {"n_neighbors":range(1,21,1)}
KNN = sklearn.neighbors.KNeighborsClassifier()

#Grid Search
KNNGrid = GridSearchCV(KNN,KNNParameters)
KNNGrid.fit(Wxtr,Wytr.ravel())
results = pd.DataFrame(KNNGrid.cv_results_)
print("Best KNN value: ",KNNGrid.best_estimator_.n_neighbors)
print("Score: ",KNNGrid.best_score_)

#Results of the GridSearch
results.iloc[:,2:7]

/usr/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


Best KNN value:  20
Score:  0.492291880781


,mean_test_score,mean_train_score,param_n_neighbors,params,rank_test_score
0,0.430370,1.000000,1,{'n_neighbors': 1},20
1,0.444245,0.831324,2,{'n_neighbors': 2},18
2,0.438078,0.787000,3,{'n_neighbors': 3},19
3,0.455550,0.721872,4,{'n_neighbors': 4},17
4,0.460946,0.701954,5,{'n_neighbors': 5},16
5,0.467112,0.676777,6,{'n_neighbors': 6},15
6,0.470195,0.661361,7,{'n_neighbors': 7},14
7,0.474049,0.647231,8,{'n_neighbors': 8},13
8,0.477133,0.637723,9,{'n_neighbors': 9},12
9,0.486639,0.624747,10,{'n_neighbors': 10},7


In [15]:
#Cross Validation of KNN- Folds=5
KNNClf = sklearn.neighbors.KNeighborsClassifier(n_neighbors=3)
CVKNN =  cross_val_score(KNNClf,Wxtst,Wytst.ravel(),cv=5)
CVKNN

/usr/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


array([ 0.44607843,  0.36945813,  0.5049505 ,  0.41708543,  0.42424242])

In [16]:
#Results on Test data of the best Hyperparameter found

KNNClf.fit(Wxtr,Wytr.ravel())
KNNClf.score(Wxtst,Wytst)

0.52882703777335982

### Iris Data - Desicion Tree

In [39]:
TreeParam ={'min_samples_split' : range(2,20,1),'max_depth': range(1,10,2)}
Tree_clf  = sklearn.tree.DecisionTreeClassifier()

#Grid Search 
#Hyperparameters: Split per leaf, and depth
Tree_grid = GridSearchCV(Tree_clf,TreeParam)
Tree_grid.fit(Ixtr,Iytr)

results = pd.DataFrame(Tree_grid.cv_results_)
print("Best split value: ",Tree_grid.best_estimator_.min_samples_split,"Best Depth value: ",Tree_grid.best_estimator_.max_depth)
print("Score: ",Tree_grid.best_score_)
results.iloc[:,2:8]



Best split value:  2 Best Depth value:  3
Score:  0.951219512195


,mean_test_score,mean_train_score,param_max_depth,param_min_samples_split,params,rank_test_score
0,0.682927,0.682895,1,2,"{'max_depth': 1, 'min_samples_split': 2}",73
1,0.682927,0.682895,1,3,"{'max_depth': 1, 'min_samples_split': 3}",73
2,0.682927,0.682895,1,4,"{'max_depth': 1, 'min_samples_split': 4}",73
3,0.682927,0.682895,1,5,"{'max_depth': 1, 'min_samples_split': 5}",73
4,0.682927,0.682895,1,6,"{'max_depth': 1, 'min_samples_split': 6}",73
5,0.682927,0.682895,1,7,"{'max_depth': 1, 'min_samples_split': 7}",73
6,0.682927,0.682895,1,8,"{'max_depth': 1, 'min_samples_split': 8}",73
7,0.682927,0.682895,1,9,"{'max_depth': 1, 'min_samples_split': 9}",73
8,0.682927,0.682895,1,10,"{'max_depth': 1, 'min_samples_split': 10}",73
9,0.682927,0.682895,1,11,"{'max_depth': 1, 'min_samples_split': 11}",73


In [128]:
#Cross Validation

Tree_clf  = sklearn.tree.DecisionTreeClassifier(min_samples_split=2,max_depth=3)
fit = Tree_clf.fit(Ixtr,Iytr)
CVTree =  cross_val_score(Tree_clf,Ixtst,Iytst.ravel(),cv=5)
CVTree

array([ 1.        ,  0.83333333,  1.        ,  0.4       ,  1.        ])

In [127]:
#Fit on test data

fit = Tree_clf.fit(Ixtr,Iytr)
fit.score(Ixtst,Iytst)

0.85185185185185186

### Wine Data - Desition Tree

In [17]:
TreeParam ={'min_samples_split' : range(2,20,1),'max_depth': range(1,10,2)}
Tree_clf  = sklearn.tree.DecisionTreeClassifier()

#Grid Search 
#Hyperparameters: Split per leaf, and depth
Tree_grid = GridSearchCV(Tree_clf,TreeParam)
Tree_grid.fit(Wxtr,Wytr)

results = pd.DataFrame(Tree_grid.cv_results_)
print("Best split value: ",Tree_grid.best_estimator_.min_samples_split,"Best Depth value: ",Tree_grid.best_estimator_.max_depth)
print("Score: ",Tree_grid.best_score_)
results.iloc[:,2:8]


/usr/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


Best split value:  2 Best Depth value:  3
Score:  0.485354573484


,mean_test_score,mean_train_score,param_max_depth,param_min_samples_split,params,rank_test_score
0,0.443988,0.463783,1,2,"{'max_depth': 1, 'min_samples_split': 2}",73
1,0.443988,0.463783,1,3,"{'max_depth': 1, 'min_samples_split': 3}",73
2,0.443988,0.463783,1,4,"{'max_depth': 1, 'min_samples_split': 4}",73
3,0.443988,0.463783,1,5,"{'max_depth': 1, 'min_samples_split': 5}",73
4,0.443988,0.463783,1,6,"{'max_depth': 1, 'min_samples_split': 6}",73
5,0.443988,0.463783,1,7,"{'max_depth': 1, 'min_samples_split': 7}",73
6,0.443988,0.463783,1,8,"{'max_depth': 1, 'min_samples_split': 8}",73
7,0.443988,0.463783,1,9,"{'max_depth': 1, 'min_samples_split': 9}",73
8,0.443988,0.463783,1,10,"{'max_depth': 1, 'min_samples_split': 10}",73
9,0.443988,0.463783,1,11,"{'max_depth': 1, 'min_samples_split': 11}",73


In [19]:
#Cross Validation

Tree_clf  = sklearn.tree.DecisionTreeClassifier(min_samples_split=2,max_depth=3)
fit = Tree_clf.fit(Wxtr,Wytr)
CVTree =  cross_val_score(Tree_clf,Wxtst,Wytst.ravel(),cv=5)
CVTree

/usr/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


array([ 0.49019608,  0.53694581,  0.51485149,  0.47236181,  0.52525253])

In [20]:
#Fit on test data
#The result is slightly better in test data than in trial data 
fit = Tree_clf.fit(Wxtr,Wytr)
fit.score(Wxtst,Wytst)

0.5188866799204771

### Final Results

In [24]:
General_Results = pd.DataFrame({ '1 Test' :("KNN","KNN","Tree","Tree"),
                                '2 Data Set': ("Iris", "Wine", "Iris", "Wine"),
                                '3 Best Hyperparameter' : ("KNN = 3","KNN = 20","Split: 2, Depth: 3","Split: 2, Depth: 3"),
                                '4 Score Grid S' : (0.9675,0.4923,0.9512,0.4853),
                                '5 Score on Test' : (0.9559,0.5288,0.8518,0.5188) })
General_Results
#It is observed that for both hyperparameters, the best results are achieved with the KNN test

,1 Test,2 Data Set,3 Best Hyperparameter,4 Score Grid S,5 Score on Test
0,KNN,Iris,KNN = 3,0.9675,0.9559
1,KNN,Wine,KNN = 20,0.4923,0.5288
2,Tree,Iris,"Split: 2, Depth: 3",0.9512,0.8518
3,Tree,Wine,"Split: 2, Depth: 3",0.4853,0.5188


# Bonus: Recommender system using similarity measures (10 Points)

Recommender Datasets: You can use one of the two datasets ( or optionally, both datasets).
    1. Movielens 100k dataset D1: Rating prediction dataset (rating scale 1-5). 
    http://grouplens.org/datasets/movielens/100k/
    2. Movielens 1m D2 : Rating prediction dataset (rating scale 1-5).
    http://grouplens.org/datasets/movielens/1m/
    3. The RMSE score for rating prediction is available at Mymedialite website 
    http://www.mymedialite.net/examples/datasets.html

#### Data Brief:
MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
        * 100,000 ratings (1-5) from 943 users on 1682 movies. 
        * Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998.

#### Task 1
In this task you are required to build a recommender system based on KNN. You will be required to

    • As usual, split your data into train and test sets.  


In [401]:
Titles = ["user id","item id","rating","timestamp"]
Data = pd.read_table("/home/salvatore/Downloads/ml-100k/u.data",names=Titles,sep="\t")
print("Original Data shape:",Data.shape)
Data.head()

Original Data shape: (100000, 4)


,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [395]:
#A pivot table allows the reconstruction of the data.
#For a user based rank prediction, the Rows are Users and the columns -"Heads" are the items

Du = pd.pivot_table(Data,values="rating",index="user id",columns="item id",fill_value=0)
print("Data Frame shape:", Du.shape)
Du.head()

Utr, Utst = split(Du,0.8)

Data Frame shape: (943, 1682)


item id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,,,,,,,
1,5,3,4,3,3,5,4,1,5,3,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,4,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [522]:
def CosineDist(A,B):
    dot_product = np.dot(A,B)
    NormA = np.linalg.norm(A)
    NormB = np.linalg.norm(B)
    return dot_product/(NormA*NormB)

def KNNCosine(TrainFrame,TestVector,k):
    Distances = []
    Similarity = []
    User_index = []
    for n in np.arange(len(TrainFrame)):
        D = CosineDist(TrainFrame[n],TestVector)
        if D==1:                                 #If D=1 it means that the vector is itself
            D=0
        Distances.append(D)
    for i in np.arange(k):
        max_i = np.argmax(Distances)
        max_d = Distances[max_i]
        Similarity.append(max_d)
        User_index.append(max_i)
        Distances[max_i]= 0
    return Similarity, User_index                       
    

In [346]:
#Example of a KNN Cosine algorithm. List of similarity and user index is showed
KNNCosine(Utr[0:1000],Utst[96],5)

([0.51028210922833461,
  0.4891753408560926,
  0.47855472419859357,
  0.46960898728593942,
  0.46450177168224999],
 [208, 186, 485, 195, 337])

#### • User KNN cosine: 
• Using k nearest neighbor users for a given query (user and item pair) and predict the rating.
    
    Note: that you have to modify your KNN algorithm implementation in Exercise 1 for User based KNN using cosine 
    similarity.
    
• Calculate Test RMSE and compare it with results presented at Mymedialite.

• Perform a hyperparameter search to get an optimal value of K.

#### Note: In the algorithm when looking for the k-NN of a column,  a new temporaly matrix is created for searching similarity only on users/articles that have a rank; in this way similarity on empty cells is avoided.

In [613]:
def RatingPredicter(TrainData,Index,Head,k):
    j = []
    TestVector = TrainData[Index-1]
    #NZero is a matrix to avoid K-NN with empty values on the column of interest
    NZero= np.array([row for row in TrainData if row[(Head-1)]>0])
    Sim,Index_i = KNNCosine(NZero,TestVector,k)
    for i in np.arange(k):
        u = Index_i[i]
        m = (NZero[u:(u+1):,(Head-1):Head]-NZero[u].mean())*Sim[i]
        j.append(m)
    Rating = TestVector.mean()+ np.sum(j)/np.sum(np.abs(Sim))
    print("Value in Matrix:",TrainData[Index-1:Index:,Head-1:Head].ravel())      #Blocked in main program
    print("Predicted: ",Rating)                                                  #Blocked in main program
    return Rating

In [605]:
# Example: Shows the actual value in the matrix vs the calculated one using Cosine similarity

RatingPredicter(Utr,18,400,15)

Value in Matrix: [0]
Predicted:  2.45345928562


2.4534592856224395

In [621]:
def RMSE_Score(TrainData,k):
    RMSE = []
    for x in np.arange(len(TrainData)):
        for n in np.arange(len(TrainData.T)):
            if TrainData[x-1:x:,n-1:n]!=0:
                y = TrainData[x-1:x:,n-1:n].ravel()
                ŷ = RatingPredicterByUser(TrainData,x,n,k)
                Error = (y-ŷ)**2
                RMSE.append(Error)
    RMSE = np.nan_to_num(RMSE)
    RMSE_Train= np.sum(RMSE)/len(RMSE)
    return RMSE_Train


def Fold(Data,fold,kNN):
    Error = []
    for k in np.arange(fold):
        L = np.random.shuffle(Data)
        w=np.round(len(Data)//fold)
        if k == 0:
            tst = Data[(k*w):((k+1)*w):]
            tr  = Data[((k+1)*w):len(Data):]
        else:
            if k== np.arange(fold)[-1]:
                tst = Data[(k*w):((k+1)*w):]
                tr  = Data[0:(k*w):]
            else:
                tst = Data[(k*w):((k+1)*w):]
                tr  = np.concatenate((Data[0:(k*w):],Data[((k+1)*w):len(Data):]),axis=0)
        Accuracy = RMSE_Score(tst,kNN)
        Error.append(Accuracy)
    return np.average(Error)


def Cvalidator(Data,fold,KNNList):
    Scores = []
    for i in np.arange(len(KNNList)):
        KNN = KNNList[i]
        iScore = Fold(Data,fold,KNN)
        Label = "KNN :",KNNList[i]
        Set = [Label,iScore]
        Scores.append(Set)
    #Best Score
    Scores.sort(key=operator.itemgetter(1))
    print("Best KNN is:")
    print(Scores[0])
    return Scores


In [622]:
List = [1,2,5,10,20]
#Example of Cross validation, 5-Folds

Cvalidator(Utr,5,List)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


Best KNN is:
[('KNN :', 2), 0.78790713744156871]


[[('KNN :', 2), 0.78790713744156871],
 [('KNN :', 5), 0.82994139356953645],
 [('KNN :', 1), 0.87770760557281435],
 [('KNN :', 10), 0.88833392468197481],
 [('KNN :', 20), 0.90860636483080681]]

#### • Item KNN cosine: 
• Using k nearest neighbor items for a given query (user and item pair) and predict the rating.

    Note: that you have to modify your KNN algorithm implementation in Exercise 1 for Item based KNN using 
    cosine similarity. 

• Calculate Test RMSE and compare it with results presented at Mymedialite.

• Perform a hyperparameter search to get an optimal value of K.

In [608]:
#A pivot table allows the reconstruction of the data.
#For a item based rank prediction, the Rows are Items and the columns -"Heads" are the users
#Then the procedure of similarity and Crossvalidation is the same as the user-base

Di = pd.pivot_table(Data,values="rating",index="item id",columns="user id",fill_value=0)
print("Data Frame shape:", Du.shape)
Itr, Itst = split(Di,0.8)

Di.head()

Data Frame shape: (943, 1682)


user id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
item id,,,,,,,,,,,,,,,,,,,,,
1,5,4,0,0,4,4,0,0,0,4,...,2,3,4,0,4,0,0,5,0,0
2,3,0,0,0,3,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,5
3,4,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,0
4,3,0,0,0,0,0,5,0,0,4,...,5,0,0,0,0,0,2,0,0,0
5,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [615]:
#Example of Rating Predict for a item-based 
RatingPredicter(Itr,4,1,15)

Value in Matrix: [3]
Predicted:  3.98935466897


3.9893546689652197

In [623]:
#Cross validator for a item-based 
Cvalidator(Itr,5,List)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


Best KNN is:
[('KNN :', 2), 0.70256067689953339]


[[('KNN :', 2), 0.70256067689953339],
 [('KNN :', 5), 0.7485395902361287],
 [('KNN :', 10), 0.81460032709764396],
 [('KNN :', 1), 0.83511180880486724],
 [('KNN :', 20), 0.85592085374758065]]

• Finally, present your results in a tabular form i.e. listing methods, hyper-parameters and Test RMSE scores.

• Hints: If you have a less powerful machine you can use movielens 100k dataset (or sub sample of it). 

Read papers in Annex section to learn more about User-KNN and Item-KNN for recommender system. You have to implement this yourself and you cannot use scikit-learn or anyother off-the-self softwares/implementations.


In [30]:
General_Results = pd.DataFrame({ '1 Test' :("User Based","Item Based","User Based MediaLite","Item Based MediaLite"),
                                '2 Data Set': ("100 k","100 k","100 k","100 k"),
                                '3 Distance Measure': ("Cosine","Cosine","Cosine","Cosine"),                                
                                '4 Best Hyperparameter' : ("KNN = 2","KNN = 2","KNN = 40", "KNN=40"),
                                '5 RMSE' : (0.7880,0.7026,0.7370,0.7270) })
General_Results
#It is observed that a lower RMSE is achieved using a Item Based approach.
#The values are similar to the ones presented on MyMediaLite

,1 Test,2 Data Set,3 Distance Measure,4 Best Hyperparameter,5 RMSE
0,User Based,100 k,Cosine,KNN = 2,0.7880
1,Item Based,100 k,Cosine,KNN = 2,0.7026
2,User Based MediaLite,100 k,Cosine,KNN = 40,0.7370
3,Item Based MediaLite,100 k,Cosine,KNN=40,0.7270


http://mymedialite.net/examples/datasets.html

<img src ="RMSE.png"/>